# Session 4 - Unsupervised Learning of Multichannel signal data 

#### :: *도 형록*

## <u>강의 내용 정리</u>

### 1. Multi Channel Signal Data & its backgrounds

* Multi-Channel Signal Data - 뇌파, 심전도, Human motion recognition, Automobile motion recognition등 다수의 센서가 동시간대에 출력하는 시계열 데이터

> **특성**
>
> - 하나의 데이터보다, 데이터의 패턴, 즉, 어떻게 변화해 가는지에 실제 관심이 있다.
>
> - 당연히 시간의 흐름이 고려가 되어, 분석이 진행되어야 한다.
>
> - 데이터 하나하나가 독립적이지 않고, 다른 데이터들과 상호종속관계에 있다. 
>
> - 상황에 따라, 데이터가 같이 움직이는 순간들이 여타 순간들 보다, 더 중요하게 다루어 질 수 있다. 
>
> - **분석단위** (연속된 특정 시간동안이 분석단위가 된다. ***Time-Window***)
>
>   * 적용가능 한 것 
>
>   > • Supervised Learning - Classification, regression, CNN
>   >
>   > • Unsupervised Learning  - Anomally detection, Event points detection, Clustering, Feature- 
>   >
>   > ​                                            extraction, Autoencoder

* Backgrounds for analysis

> Sensor들이 어떤 패턴(혹은 관계의 패턴)을 가졌는지 알기 위해, Covariance matrix를 사용한다. 
>
> $\rightarrow$ Correlation Vs **Partial Correlation(conditional independency)** ; Partial correlation이 각 관계를 잘 설명해준다.(Sparse graphs are  more interpretable)
>
> * Graphical Lasso (Friedman, Hastie and Tibshirani, 2008)
>
>   $\text{Minimize}_{\theta \in \mathcal{s}_{++}} -\mathcal{l}(\Theta) + \lambda||\Theta||_{od, 1} \rightarrow$ 어느 시점에서 센서간의 관계가 바뀌는지 Detecting 

### 2. Correlation related Method

##### Time-Varying Graphical Lasso (Hallac, Park, Boyd and Leskovec, 2017)


{ time-varing : change overtime , Graphical Lasso : Sparse Inverse Covariance , Pairwise Relationship}

:: 분석단위 - window size, 핵심은 시간의 순서에 구애 받지 않고 time을 변화시키면 , 오류가 없이 분석 가능하리라 기대 (기본적으로 변화는 시간에따라 부드럽게 일어 날 것 이다. ), window안에서는 변화가 크게 없을 것이라는 가정

• Formulation 

- Static Graphical Lasso : $minimizd_{\theta \in \mathcal{S}_{++}} -l(\Theta) + \lambda||\Theta||_{od, 1}$  

  (Loss func : negaticce log likelihood + Penalty func1 : sparseness of graph)

- Time-Varying Graphical Lasso Formulation

  ($minimize_{\Theta \in \mathcal{S}^{p}_{++}}\sum^T_{t=1}[-l_t(\Theta_t) + \lambda||\Theta_t||_{od, 1}] + \beta \sum^T_{t=2}\psi(\Theta_t - \Theta_{t-1})$  Penalty function2 added(smoothness over adjacent graphs)) 

(전처리 단계에서, window size는 정의되어야 한다. 작을 수록 작은 변화를 detacking할 수 있으나, 연산량이 급격히 증가한다. )

:: $\lambda$ ; 어느 정도까지 correlation이 달라야 다른 것으로 볼 것 인가 ? 

:: $\beta$ ; 시간에 따라, 얼마나 변하게 할 것 인 가 ? (작을 수록 시간에 따라 많이 변할 것이다. )

:: $\psi(\Theta_t - \Theta_{t-1})$ 어떤 관계를 위주로 볼 것 인가? 

* ###### 효과

  - labels이 없는 Multivariate간의 관계 변화를 감지 할 때 효과적이었다. 
  - 500개의 채널 시그널 데이터까지 커버가능(Scaliablity )
  - 한계 ;
    - window size 내에서 종속 된다. 
    - 전체 데이터를 다 구성해놓고 분석 ( Online으로 불가 )

##### Toeplitz Inverse Covariance-based Clustering 

{Toeplitz : 행렬의 종류, Inverse Covariance : Partial Corr., Relationship between Channels}

¶  $Toeplitz = \begin{bmatrix} a_0 & a_{-1} & a_{-2} & \cdots & a_{-n} \\ a_1 & a_0 & a_{-1}& \cdots &a_{-(n-1)}\\ a_2 & a_1 & a_0 & a_{-1} & \cdots\\  \vdots & a_2 & a_1 & a_0 & \cdots \\ a_n & \cdots & \cdots & \cdots & a_0\end{bmatrix}$ 이 와 같이 우하향 벡터 요소끼리 값이 같은 행렬을 뜻한다. 

¶ Inverse Covariance : Cluster 마다의 Sentroider의 역할을 한다. 

###### Cross-Correlation network

:: 시점에 따른 'Time log' (한 시점에 대한 것만 다루는 Markov random field( 바로 앞 과의 차이)가 아닌 Multilayer를 다룬다. )

:: Time lag correlation이라고도 불린다. 여러시점을 같이 고려한다는 점에서, 분석단위인 window size를 잘게 쪼갤 수 있다. 			$\rightarrow$ 어떤 채널끼리, 시차를 두고 영향을 받는 경우를 detecting할 수 있다.  

* **Formulation** 

$minimize_{\Theta \in T, P} \sum^{K}_{k=1}[\lambda||\Theta_k||_1 + \sum_{X_t \in P_k}(-l(\Theta_k;X_t) + \beta \mathcal{I}(X_{t-1} P_k) )$

:: window size : 시점 산정에 따라, Covariance 형태가 달라짐, 크게 줄수록 다양한 변화 감지, 연산량은 증가

:: $\lambda||\Theta_k||_1$ - Penalty func for sparseness of graph, $\lambda$를 크게 주면, 아주 중요한 변화만 남김

:: $-l(\Theta_k;X_t) $ - Loss func for negatice log-likelihood

:: $\beta \mathcal{I}(X_{t-1} P_k)$ - Penalty func for temporal consistency ( 인접한 데이터를 같은 Cluster에 할당하기 위해서 )

; $\beta$ 가 클수록 인접한 데이터를 같은 Cluster에 할당한다. 

$\rightarrow$ 이것의 장점은 클러스터의 특성도 알 수 있다는 점이다.  (각각의 Cluster에서 어떤 sensor가 중요한지)

### 3. Deeplearning applied 

#### • Convolutional Autoencoder for abnormal detection 

 :: Spatially-local correlation : 인접한 숫자들은 높게 관련있을 것이다. 

 :: Location invariant features : 중요한 정보들은 고정된 위치에서만 발견되는 것이 아니다. 

$\rightarrow$ convolution filter는 시그널 데이터의 지역적인 패턴을 잘 찾을 수 있을 것이다. 

* **Sturcture** 

  기존의 Convolutional Autoencoder와 동일한 구조를 가지고 있다. 

  INPUT - Encoder( conv - pooling x 2) - Embedding - Decoder(TransPosed Conv - Unpooling x 2) - Reconstucted

  :: _Loss Function_  : $ minimize \dfrac{1}{n}\sum_{i=1}^n||x_i - \hat{x}_i||_2^2$ (원래의 시그널의 복구가 목적)

   -> Initializer : Xavier, Optimizer : Adam, Act. func : Relu/leaky relu, BatchNormalization, No dense layer

  _tip! Autoencoder는 Normal한 데이터가 있다면,  overfitting 하는 것이 실제 기능시에 더 좋을 수 있다._

* Usage 

  With Reconsturcted signal : Anomaly detection 

  With Feature extraction : Use bottleNeck part of autoencoder ( clustering can be conducted furthermore)

  -> Word2Vec처럼 유사한 시그널은 유사한 Embedding Vector를 가진다고 한다. 이유는 안 가르쳐주심. ( time window간의 유사도를 구하는 것은 어려운 작업이었다. )

* 한계점

  **Anomality 에 대한 Threshold를 설정하는 것이 어려웠다. **

####• Convolutional Variational Autoencoder 

( Conv-Autoencoder의 한계점을 어느정도 극복. ) 

:: 구조는 Convolutional Autoencoder와 같으나, 중간의 Embedding 에서 값을 그대로 쓰는 것이 아니라, Gaussian으로 랜덤 샘플링을 해준다. -> Generative Model의 특성(random한 값에서, Reconstructed signal 을 출력하는 것)을 가진다. 

* **Loss Function**

  $ minimize \rightarrow E_{q_{\phi}(z|x_i)}[\log{p_\theta(x_i, z)}] + D_{KL}(q_{\phi}(z|x_i)||p_{\theta}(z|x_i))$

  Reconsturct와 embedding 모두 잘하도록 학습시킨다. 

  중간의 Embedding layer를 **Latent variable** 라 부르는데, 이를 어떤 분포로 주냐에 따라 다양하게 만들 수 있다. 

  (GAN 전까지 가장 성능이 좋았다.)

* **Threehold** - Threshold를 설정하기 비교적 용이 했다. 

  $ Anomaly Score = 1 - \text{E}_{q\theta(z|x_i)}[p_{\phi}(x_i|z)]$

  $ \text{E}_{q\theta(z|x_i)}[p_{\phi}(x_i|z)]$ : Reconstruction Probability 


